In [1]:
import pandas as pd
import numpy as np
import alphalens
import matplotlib.pyplot as plt
import statsmodels.api as sm
import matplotlib
from dateutil.relativedelta import relativedelta

D:\Anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
D:\Anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.chdir('../')

In [4]:
matplotlib.rcParams['font.sans-serif']=['Times New Roman'] 

#### 1.数据读取

In [5]:
#股票数据，并求出股票的log return用以
stk_close = pd.read_csv("./Data/StkClose.csv",index_col = 0)

In [6]:
stk_close.index = pd.to_datetime(stk_close.index)

In [7]:
filled_stk_close = stk_close.asfreq("D").ffill()

In [8]:
filled_stk_lr = np.log(filled_stk_close).diff(axis = 0)#可以使用log return作差，因为主要还是判断正负号

In [9]:
# 基金数据，并分类到股票型基金
nv_2010to2014 = pd.read_csv("./Data/AdjustedNav(2010to2014).csv",index_col = 0)
nv_2015to2018 = pd.read_csv("./Data/AdjustedNav(2015to2018).csv",index_col = 0)
nv_2019to2022 = pd.read_csv("./Data/AdjustedNav(2019to2022).csv",index_col = 0)

In [10]:
nv = pd.concat([nv_2010to2014,nv_2015to2018,nv_2019to2022])

In [11]:
nv.index = pd.to_datetime(nv.index)

In [12]:
filled_nv = nv.asfreq("D").ffill()

In [13]:
filled_nv_return = nv.pct_change(axis = 0)#简单收益率
filled_nv_lr = np.log(filled_nv).diff(axis = 0)#对数收益率，log return

In [14]:
# 市场收益率
rm_stk_port = pd.read_excel("./Data/股票型基金指数.xlsx",index_col = "date")
rm_citic = pd.read_excel("./Data/中信500宽基指数.xlsx",index_col = "date")

filled_rm_stk_port = rm_stk_port.asfreq("D").ffill()
filled_rm_citic = rm_citic.asfreq("D").ffill()

filled_rm_stk_lr = np.log(filled_rm_stk_port).diff()#对数的偏股型基金指数收益率
filled_rm_citic_lr = np.log(filled_rm_citic).diff()#对数的中证500宽基指数收益率

In [15]:
# 导入因子数据
factor_info = pd.read_csv("./Data/FactorInfo.csv",index_col = 0)

In [16]:
factor_info.index = pd.to_datetime(factor_info.index)
filled_factor_info = factor_info.asfreq("D").ffill()

In [17]:
#导入成立日期、基金规模、带股基金类型等限定数据
stkfund_info = pd.read_excel('./Data/StkMutualFundInfo.xlsx',index_col = 0)
stkfund_scale = pd.read_excel('./Data/StkMutualaFundScale.xlsx',index_col = 0)

In [18]:
# 月初月末表
month_begin_end = pd.read_excel("./Data/MonthBeginEnd.xlsx")

#### 2.获取基金每个季度的alpha因子

In [19]:
filled_factor_info["MONTH"] = filled_factor_info.index.to_period('m')

In [20]:
filled_factor_info["DATE"] = filled_factor_info.index

In [21]:
backtest_month = filled_factor_info.groupby(["MONTH"]).last().DATE

In [22]:
alpha = pd.DataFrame(index = backtest_month,columns = nv.columns)

In [23]:
start_date = pd.to_datetime("2013-12-31")
end_date = pd.to_datetime("2022-03-31")

In [24]:
trade_date = pd.read_excel("./Data/TradeDate.xlsx")

In [25]:
stkfund_list = stkfund_info.query("IS_INITIAL == '是' and IS_FOF == '否'").index.tolist()

In [26]:
trade_filled_nv_lr = filled_nv_lr.loc[list(set(filled_nv_lr.index)&set(trade_date.DATE))].sort_index()

In [27]:
for month in backtest_month[backtest_month >= start_date]:
    print(month,month - relativedelta(months = 3)) # backtest season start and end
    for fund in filled_nv_lr.loc[:,set(stkfund_list) & set(filled_nv_lr.columns)]: # mutual fund with stock heavyweight
        tmp_return = trade_filled_nv_lr.loc[month - relativedelta(months = 3):month,fund] # last season 
        tmp_return.dropna(inplace = True)
        if len(tmp_return) < 45: # not enough to fit 
            continue
        tmp_factor = factor_info.loc[month - relativedelta(months = 3):month] # last season
        tmp_factor = sm.add_constant(tmp_factor) 

        # paramaters estimation
        est = sm.OLS(tmp_return-tmp_factor.RF,tmp_factor.drop("RF",axis = 1)) # factor model fit
        est = est.fit()
        tmp_alpha = est.params.const # alpha
        tmp_pvalue = est.pvalues.const # p-value
#         if tmp_pvalue<=0.05 and tmp_alpha > 0:
#             alpha.at[month,fund] = tmp_alpha # store alpha, if and only if significant and bigger than zero            
#             print(f"{month},{fund},{tmp_alpha},{tmp_pvalue<=0.05} finished") # significant at 90% level
#         if  tmp_alpha > 0:
#             alpha.at[month,fund] = tmp_alpha # store alpha, if and only if significant and bigger than zero            
#             print(f"{month},{fund},{tmp_alpha},{tmp_pvalue<=0.05} finished") # significant at 90% level
        alpha.at[month,fund] = tmp_alpha

2013-12-31 00:00:00 2013-09-30 00:00:00
2014-01-31 00:00:00 2013-10-31 00:00:00
2014-02-28 00:00:00 2013-11-28 00:00:00
2014-03-31 00:00:00 2013-12-31 00:00:00
2014-04-30 00:00:00 2014-01-30 00:00:00
2014-05-31 00:00:00 2014-02-28 00:00:00
2014-06-30 00:00:00 2014-03-30 00:00:00
2014-07-31 00:00:00 2014-04-30 00:00:00
2014-08-31 00:00:00 2014-05-31 00:00:00
2014-09-30 00:00:00 2014-06-30 00:00:00
2014-10-31 00:00:00 2014-07-31 00:00:00
2014-11-30 00:00:00 2014-08-30 00:00:00
2014-12-31 00:00:00 2014-09-30 00:00:00
2015-01-31 00:00:00 2014-10-31 00:00:00
2015-02-28 00:00:00 2014-11-28 00:00:00
2015-03-31 00:00:00 2014-12-31 00:00:00
2015-04-30 00:00:00 2015-01-30 00:00:00
2015-05-31 00:00:00 2015-02-28 00:00:00
2015-06-30 00:00:00 2015-03-30 00:00:00
2015-07-31 00:00:00 2015-04-30 00:00:00
2015-08-31 00:00:00 2015-05-31 00:00:00
2015-09-30 00:00:00 2015-06-30 00:00:00
2015-10-31 00:00:00 2015-07-31 00:00:00
2015-11-30 00:00:00 2015-08-30 00:00:00
2015-12-31 00:00:00 2015-09-30 00:00:00


In [28]:
alpha.to_csv("./Output/alpha_log_return(all).csv")